# Models

In this notebook, the data is uploaded from Kaggle dataset and stored in a Google Coud Storage bucket to be used with TPUs. The data itself is in a tfrec format for convenient distribution to the TPU cores.

The final layers of three pretrained networks are trained on the data and the results are compared in the next notebook. 


## Import libraries

In [ ]:
import math, re, os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

print("Tensorflow version " + tf.__version__)


## Distribution Strategy

In [ ]:
# Detect TPU, return distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Running on TPU ", tpu.master())
except ValueError:
    tpu = None
    
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
    
print("REPLICAS: ", strategy.num_replicas_in_sync)


## Load Data

In [ ]:
from kaggle_datasets import KaggleDatasets

GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(GCS_DS_PATH) 

In [ ]:
import os
dirs = os.listdir("../input")

for file in dirs:
    print(file)

The Kaggle data is split into a labeled training set, a labeled validation set, and an unlabeled test set. The unlabeled set is used for competition submission and will not be used here. Instead, the validation set will be used as the test set and the training set will be split for validation.

In [ ]:

IMAGE_SIZE = [331, 331]
GCS_PATH =  GCS_DS_PATH + '/tfrecords-jpeg-331x331'
AUTO = tf.data.experimental.AUTOTUNE

validation_split = 0.25
data_files = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
split = len(data_files) - int(len(data_files) * validation_split)

TRAINING_FILENAMES = data_files[:split]
VALIDATION_FILENAMES = data_files[split:]
TEST_FILENAMES =  tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')

CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102


def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset




## Helper Functions

Functios to augment and retrieve data

In [ ]:
def zoom(x: tf.Tensor) -> tf.Tensor:
    #Zoom Augmentation
    
    scales = list(np.arange(0.8, 1.2, 0.02))
    boxes = np.zeros((len(scales), 4))
    
    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]
        
    def random_crop(img):
        #Create different crops
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_indices=np.zeros(len(scales)), crop_size=(331, 331) )
        return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]
    
    choice =  tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)
    
    #Applies 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))


def rotate(x: tf.Tensor) -> tf.Tensor:
    
    image = tf.cond(tf.greater(tf.random.uniform([]), 0.5), lambda: tf.image.rot90(x), lambda: x)
    return image

def data_augment(image, label):
    #Execute a series of random image augmentations 
    
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_hue(image, 0.08)
    image = tf.image.random_saturation(image, 0.6, 1.6)
    image = tf.image.random_brightness(image, 0.05)
    image = tf.image.random_contrast(image, 0.7, 1.3)
    image = zoom(image)
    image = rotate(image)
    return image, label

REPEAT_DATA_AUGMENT = 12

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES)
    dataset = dataset.repeat(REPEAT_DATA_AUGMENT)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

# Learning Rate Schedule for Fine Tuning #
def exponential_lr(epoch,
                   start_lr = 0.00001, min_lr = 0.00001, max_lr = 0.00005,
                   rampup_epochs = 5, sustain_epochs = 0,
                   exp_decay = 0.8):

    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr = ((max_lr - start_lr) /
                  rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        # exponential decay towards min_lr
        else:
            lr = ((max_lr - min_lr) *
                  exp_decay**(epoch - rampup_epochs - sustain_epochs) +
                  min_lr)
        return lr
    return lr(epoch,
              start_lr,
              min_lr,
              max_lr,
              rampup_epochs,
              sustain_epochs,
              exp_decay)



NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES) * REPEAT_DATA_AUGMENT
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print('Dataset: {} training images, {} validation images, {} test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))


## Get Datasets Ready for Training

In [ ]:
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

ds_train = get_training_dataset()
ds_valid = get_validation_dataset()
ds_test = get_test_dataset()

print("Training:", ds_train)
print ("Validation:", ds_valid)
print("Test:", ds_test)

In [ ]:
print(BATCH_SIZE)

## Define Models

### VGG

In [ ]:

# Define training epochs
EPOCHS = 30
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

with strategy.scope():
    pretrained_vgg_model = tf.keras.applications.vgg16.VGG16(
        weights='imagenet',
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    for layer in pretrained_vgg_model.layers:
        if layer.name == 'block14_sepconv1' or layer.name == 'block14_sepconv2':
           layer.trainable = True
        else:
            layer.trainable = False
    
    vgg_model = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_vgg_model,
        # ... attach a new head to act as a classifier.
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])

In [ ]:
pretrained_vgg_model.summary()

In [ ]:
from tensorflow.keras import optimizers

optimizer = optimizers.Adam(lr=0.01)

vgg_model.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

vgg_model.summary()

In [ ]:


#from tensorflow.keras.callbacks import EarlyStopping

#early_stopping = EarlyStopping(patience=3, verbose=1,restore_best_weights=True)
lr_callback = tf.keras.callbacks.LearningRateScheduler(exponential_lr, verbose=True)

vgg_history = vgg_model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback]
)

In [ ]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
vgg_model.save('./vggmodel', options=save_locally)


## Resnet50

In [ ]:
with strategy.scope():
    pretrained_resnet_model = tf.keras.applications.resnet50.ResNet50(
        weights='imagenet',
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    for layer in pretrained_resnet_model.layers:
        if layer.name == 'block14_sepconv1' or layer.name == 'block14_sepconv2':
           layer.trainable = True
        else:
            layer.trainable = False
    
    resnet_model = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_resnet_model,
        # ... attach a new head to act as a classifier.
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])

In [ ]:
pretrained_resnet_model.summary()

In [ ]:
resnet_model.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

resnet_model.summary()

In [ ]:

resnet_history = resnet_model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback]
)

In [ ]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
resnet_model.save('./resnetmodel', options=save_locally)


## InceptionNet V3

In [ ]:
with strategy.scope():
    pretrained_inception_model = tf.keras.applications.inception_v3.InceptionV3(
        weights='imagenet',
        include_top=False ,
        input_shape=[*IMAGE_SIZE, 3]
    )
    for layer in pretrained_inception_model.layers:
        if layer.name == 'block14_sepconv1' or layer.name == 'block14_sepconv2':
           layer.trainable = True
        else:
            layer.trainable = False
    
    inception_model = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_inception_model,
        # ... attach a new head to act as a classifier.
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])

In [ ]:
pretrained_inception_model.summary()

In [ ]:
inception_model.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

inception_model.summary()

In [ ]:
inception_history = inception_model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback]
)

In [ ]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
inception_model.save('./inceptionmodel', options=save_locally)